# Generating ImageNet Images

Here we go through the steps to get a trained model to generate ImageNet images.

In [1]:
%cd ..

/home/jo3/p/features2image_diffusion


/home/jo3/p/features2image_diffusion/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning:

using dhist requires you to install the `pickleshare` library.



Our dataset, ImageNetSet will return features, images and labels, therefore we need to tell it the location of the features and the location of the stored images and labels.  
We give an additional argument that gets passed to the torchvision.transform.Resize function, to reduce the size of the outputted images.

In [2]:
from features2image_diffusion.data import ImageNetSet

dataset = ImageNetSet(
    "../sparse_autoencoder/run/1ae3fe65/train",
    "../sparse_autoencoder/res/imagenet/train",
    image_size=64,
)

Loading dataset from disk:   0%|          | 0/294 [00:00<?, ?it/s]

In [3]:
features, image, label = dataset[0]
print(features.shape, image.shape, type(label))

num_features = features.shape[0]
n_channels, _, img_len = image.shape
num_features, n_channels, img_len

torch.Size([40768]) torch.Size([3, 64, 64]) <class 'int'>


(40768, 3, 64)

In [4]:
from features2image_diffusion.unet import load_ddpm

ddpm = load_ddpm(
    "run/f2a2e4ac/model/epoch-39.pth",
    n_classes=num_features,
    n_channels=n_channels,
    img_len=img_len,
    hidden_size=128,
    diffusion_steps=400,
    device="cpu",
).eval()

In [5]:
import torch
from torch import Tensor

with torch.no_grad():
    generations, _ = ddpm.sample(
        context=Tensor(features),
        n_sample=3,
        size=(3, 64, 64),
        device="cpu",
        verbose=True,
        store=False,
    )

generations.shape

RuntimeError: [enforce fail at alloc_cpu.cpp:83] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 256490078208 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
from jo3mnist.vis import to_img
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick